In [1]:
#r "nuget:Serilog.Formatting.Compact.Reader,1.0.3"
#r "nuget:System.Linq.Async,4.0.0"

Installed package System.Linq.Async version 4.0.0

Installed package Serilog.Formatting.Compact.Reader version 1.0.3

In [1]:
using System.IO;
using Serilog.Events;
using Serilog.Formatting.Compact.Reader;
using System.Collections.Generic;
using System.Linq;

In [1]:
public async IAsyncEnumerable<PlotDatum> GetBestFrameCount()
{
    var compactFile = new FileInfo(@"C:\Users\vaido\SourceCode\MrBabyData\FrameRateAnalysis\RogueTD_PerformanceMetricsFrameRates.clef");
    using(var stream = compactFile.OpenText())
    {
        while(stream.Peek() > 0){
            var logevent = Serilog.Formatting.Compact.Reader.LogEventReader.ReadFromString(await stream.ReadLineAsync());
            yield return new PlotDatum{
                timeStamp = logevent.Timestamp,
                bestFrameCount = float.Parse(logevent.Properties["bestDuration"].ToString())
            };
        }
    }
}

public struct PlotDatum{
    public DateTimeOffset timeStamp {get; set;}
    public float bestFrameCount {get; set;}
}

var plotDatums = await GetBestFrameCount().ToListAsync();

In [1]:
#!javascript
notebookScope.plot = (sgvSelector) => {
    let dtreeLoader = interactive.configureRequire({
        paths: {
            d3: "https://d3js.org/d3.v6.min"
        }
    });

    dtreeLoader(["d3"], function (d3) {

    var fakeData = [
        {
          x: 0,
          y : 0
        },
        {
          x : 20,
          y : 50
        },
        {
          x : 100,
          y : 15
        }
      ]

    var buildFrameRatePlot = (svg, width, height, data) => {
        var layoutMeasures = {
          width : width,
          height : height,
          plotPadding: [24,24,24,24], // top, right, bottom, left
          get plotWidth()
          {
            return layoutMeasures.width - layoutMeasures.plotPadding[1] - layoutMeasures.plotPadding[3]
          },
          get plotHeight()
          {
            return layoutMeasures.height - layoutMeasures.plotPadding[0] - layoutMeasures.plotPadding[2]
          }
        }

        // clean clear our svg
        svg.selectAll("g").remove();

        // plot container
        let container = svg
        .append("g")
        .attr("height", layoutMeasures.plotHeight)
        .attr("width", layoutMeasures.plotWidth)
        .attr("transform", `translate(${layoutMeasures.plotPadding[0]}, ${layoutMeasures.plotPadding[3]})`);

        container.append("rect")
          .attr("height", layoutMeasures.plotHeight)
          .attr("width", layoutMeasures.plotWidth)
          .style("fill", "#F7F7F7")

        var y = d3.scaleLinear()
          .domain([0, 120])
          .range([layoutMeasures.plotHeight, 0])
        var x = d3.scaleLinear()
          .domain([0, 900])
          .range([0, layoutMeasures.plotWidth])
        var line = d3.line()
          .defined(d => !isNaN(d.x))
          .defined(d => !isNaN(d.y))
          .x(d => x(d.x))
          .y(d => y(d.y))

        var yAxis = g => g
            .attr("transform",`translate(0,0)`)
            .call(d3.axisLeft(y))
        var xAxis = g => g
            .attr("transform",`translate(0,${layoutMeasures.plotHeight})`)
            .call(d3.axisBottom(x))

        container.append("g")
            .call(yAxis);
        container.append("g")
            .call(xAxis);

        container.append("path")
          .datum(data)
          .attr("fill", "none")
          .attr("stroke", "steelblue")
          .attr("stroke-width", 1)
          .attr("d", line);
    }
    
    var svg = d3.select(sgvSelector);
    console.log(svg);
    buildFrameRatePlot(svg, 300, 500, fakeData);
    });
}

In [1]:
#!html
<svg id="dataPlot1" height=300 width=1000></svg>

#!js
notebookScope.plot("svg#dataPlot1" );